In [ ]:
from google.colab import files
files.upload()
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances_argmin
from sklearn.utils import shuffle
from time import time
from PIL import Image


"""HELPER FUNCTIONS"""

# Perform quantization - called by runner()
def quantize(img="A.PNG",
             img_quant_kmean="A_kmean.PNG", img_quant_random="A_random.PNG"):
    """
    Normalize the R/G/B histograms of B separately to match A
    :param img: The filepath (string) to the original image, with image file extension
    :param img_quant_kmean: The filepath (string) to the image after K-means quantization, with image file extension
    :param img_quant_random: The filepath (string) to the image after random quantization, with image file extension
    :returns: Nothing, but the image files are just saved in the respective filepaths
    """

    n_colors = 64

    # Load the photo
    oilspill = Image.open(img)

    # Convert to floats instead of the default 8 bits integer coding. Dividing by
    # 255 is important so that plt.imshow behaves works well on float data (need to
    # be in the range [0-1])
    oilspill = np.array(oilspill, dtype=np.float64) / 255

    # Load Image and transform to a 2D numpy array.
    w, h, d = original_shape = tuple(oilspill.shape)
    image_array = np.reshape(oilspill, (w * h, d))

    # print("Fitting model on a small sub-sample of the data")
    # t0 = time()
    image_array_sample = shuffle(image_array, random_state=0)[:1000]
    kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(image_array_sample)
    # print("done in %0.3fs." % (time() - t0))

    # Get labels for all points
    # print("Predicting color indices on the full image (k-means)")
    # t0 = time()
    labels = kmeans.predict(image_array)
    # print("done in %0.3fs." % (time() - t0))

    codebook_random = shuffle(image_array, random_state=0)[:n_colors]
    # print("Predicting color indices on the full image (random)")
    # t0 = time()
    labels_random = pairwise_distances_argmin(codebook_random,
                                              image_array,
                                              axis=0)
    # print("done in %0.3fs." % (time() - t0))

    def recreate_image(codebook, labels, w, h):
        """Recreate the (compressed) image from the code book & labels"""
        d = codebook.shape[1]
        image = np.zeros((w, h, d))
        label_idx = 0
        for i in range(w):
            for j in range(h):
                image[i][j] = codebook[labels[label_idx]]
                label_idx += 1
        return image

    # Display all results, alongside original image
    # plt.figure(1)
    # plt.clf()
    # plt.axis('off')
    # plt.title('Original image (96,615 colors)')
    # plt.imshow(oilspill)

    # plt.figure(2)
    # plt.clf()
    # plt.axis('off')
    # plt.title('Quantized image (64 colors, K-Means)')
    # plt.imshow(recreate_image(kmeans.cluster_centers_, labels, w, h))

    img1_output = recreate_image(kmeans.cluster_centers_, labels, w, h)
    img2_output = recreate_image(codebook_random, labels_random, w, h)

    # plt.figure(3)
    # plt.clf()
    # plt.axis('off')
    # plt.title('Quantized image (64 colors, Random)')
    # plt.imshow(recreate_image(codebook_random, labels_random, w, h))
    # plt.show()

    kmean_output = Image.fromarray((img1_output * 255).astype(np.uint8), "RGBA");
    kmean_output.save(img_quant_kmean)
    random_output = Image.fromarray((img2_output * 255).astype(np.uint8), "RGBA");
    random_output.save(img_quant_random)


# Split image array into R/G/B channels - called by normalize()
def split_into_rgb_channels(image):
    """
    Split the target image into its red, green and blue channels.

    :param image: The input image as an numpy array of shape (rows, columns, 3).
    :returns:
        output - three numpy arrays of shape (rows, columns) and dtype same as
         image, containing the corresponding channels.
    """
    red = image[:, :, 2]
    green = image[:, :, 1]
    blue = image[:, :, 0]
    return red, green, blue


# Perform normalization - called by runner()
def normalize(before="A.PNG", after="B.PNG", after_norm="B_norm.PNG"):
    """
    Normalize the R/G/B histograms of B separately to match A
    :param before: The filepath (string) to the before image, with image file extension
    :param after: The filepath (string) to the after image, with image file extension
    :param after_norm: The filepath (string) to the after image after normalization, with image file extension
    :returns: Nothing, but the image files are just saved in the respective filepaths
    """

    # READ IMAGES
    reference2D = cv2.imread(before)  # before - by convention this is the reference
    source2D = cv2.imread(after)  # after

    # Initialise a new size to standardise dimensions between the two images
    new_size = np.asarray(reference2D.shape)
    reference2D = cv2.resize(reference2D, (new_size[1], new_size[0])).astype(int)  # BUG FIX
    source2D = cv2.resize(source2D, (new_size[1], new_size[0])).astype(int)  # BUG FIX

    reference2D_red, reference2D_green, reference2D_blue = split_into_rgb_channels(reference2D)
    source2D_red, source2D_green, source2D_blue = split_into_rgb_channels(source2D)
    target_red = np.zeros(source2D_red.shape).astype(int)
    target_green = np.zeros(source2D_green.shape).astype(int)
    target_blue = np.zeros(source2D_blue.shape).astype(int)

    for reference2D, source2D, channel in zip(
        (reference2D_red, reference2D_green, reference2D_blue),
        (source2D_red, source2D_green, source2D_blue),
        (2, 1, 0)):

        # For the rest of the analysis, unravel into a 1D array
        source = source2D.ravel()
        reference = reference2D.ravel()

        # Get the set of unique pixel values and their corresponding indices and counts
        s_values, s_idx, s_counts = np.unique(
            source, return_inverse=True, return_counts=True)

        # We can do the same with the reference (but we don't need the r_idx for this)
        r_values, r_counts = np.unique(reference, return_counts=True)

        # Now we need to calculate the empirical cumulative distribution, scaled 0 to 1.
        # Each quantile tells us, for each unique value, what proportion of the data fall at or below that value.
        # - If matching by CDF:
        s_quantiles = np.cumsum(s_counts).astype(np.float64) / source.size
        r_quantiles = np.cumsum(r_counts).astype(np.float64) / reference.size

        # To put it simply, we need to adjust source so that the blue line
        # matches the green line as closely as possible.
        # To do this, we can linearly interpolate on the arrays. This is the core concept of the algorithm.
        # The value of the source CDF at each unique input is mapped to the reference CDF
        # (It need not be an exact equality as we use linear interpolation).
        # The reference cdf value is then used to look up the corresponding reference value
        # (interpolated between values if the match isn't exact).
        # The interpolated values have the same shape as s_values -
        # the result is the new set of source values to replace the original.
        interp_r_values = np.interp(s_quantiles, r_quantiles, r_values)

        # Now we can take our s_idx to recreate an array of the same size as the source but with new values
        # this is the "new" source2D aka. the "B" image
        if channel == 2:
            target_red = interp_r_values[s_idx].reshape((new_size[0], new_size[1]))
            target = target_red
        elif channel == 1:
            target_green = interp_r_values[s_idx].reshape((new_size[0], new_size[1]))
            target = target_green
        else:
            target_blue = interp_r_values[s_idx].reshape((new_size[0], new_size[1]))
            target = target_blue

        # We can also visually assess the quality of the histogram match
        t_values, t_counts = np.unique(target, return_counts=True)
        t_quantiles = np.cumsum(t_counts).astype(np.float64) / target.size

        # plt.plot(s_values, s_quantiles, label="Source")
        # plt.plot(r_values, r_quantiles, label="Reference")
        # plt.plot(t_values, t_quantiles, '--r', lw=2, label="Target")
        # plt.title(channel)
        # plt.legend(loc=5)
        # plt.show()

    target = cv2.merge((target_blue, target_green, target_red)).astype(int)

    # Save normalised source2D image
    cv2.imwrite(after_norm, target)


# Find vector set - called by pcakmc()
def find_vector_set(diff_image, new_size, h):
    """
    Using the difference image, finds the vector set x_d(y,x) and the mean vector of this set.

    :param diff_image: Difference image X_d = |X_2 - X_1|
    :param new_size: Dimensions of the difference image
    :param h: Dimensions of sampling (square) block of neighbours to use for flattening to obtain 1d vector
    :returns:
        vector_set - A row matrix with (m×n)/(5×5) rows, where m×n is the dimension of the diff image
        mean_vec - The mean vector of vector_set
    """
    i = 0
    j = 0
    vector_set = np.zeros((int(new_size[0] * new_size[1] / h**2), h**2))  # intialise vector_set

    # Obtain h×h *non-overlapping* blocks of the difference image
    while i < vector_set.shape[0]:
        while j < new_size[1]:
            k = 0
            while k < new_size[0]:
                block = diff_image[k:k + h, j:j + h]  # BUG FIX 6 Nov 2020 swapped j and k
                feature = block.ravel()  # flatten the block to obtain a 1d vector
                vector_set[i, :] = feature
                k = k + h
            j = j + h
        i = i + 1
    mean_vec = np.mean(vector_set, axis=0)
    vector_set = vector_set - mean_vec  # mean-normalise the vector_set
    return vector_set, mean_vec


# Find feature vector space (FVS) - called by pcakmc()
def find_FVS(EVS, diff_image, mean_vec, new_size, r):
    """
    Finds feature vector space using eigenvector space,
    by projecting the obtained eigenvectors onto to the difference image

    :param EVS: Eigenvector set from applying PCA to the vector set
    :param diff_image: Difference image X_d = |X_2 - X_1|
    :param mean_vec: Mean vector of vector_set
    :param new_size: Dimensions of the difference image
    :param r: Sampling block half-length
    :return: FVS - Feature vector set
    """
    i = r
    feature_vector_set = []
    while i < new_size[1] - r:
        j = r
        while j < new_size[0] - r:
            block = diff_image[j-r:j+r+1, i-r:i+r+1]  # BUG FIX 6 Nov 2020 swapped i and j
            feature = block.flatten()  # flatten the feature_vector_set into a 1D vector
            feature_vector_set.append(feature)
            j = j + 1
        i = i + 1
    FVS = np.dot(feature_vector_set, EVS)
    FVS = FVS - mean_vec  # mean-normalise the FVS
    return FVS


# Apply k-means clustering to FVS - called by pcakmc()
def clustering(FVS, clusters, new_size, r):
    """
    Performs k-means clustering of the FVS to obtain two clusters.
    The cluster with the lowest number of pixels (since the changed component is lesser than the changed)
    is assigned to the least_index.

    :param FVS: Feature vector set
    :param clusters: Number of clusters used
    :param new_size: Dimensions of the difference image
    :param r: Sampling block half-length
    :returns:
        least_index - Index of identified cluster with fewest pixels
        change_map - Final resulting image that displays detected changes
    """
    kmeans = KMeans(clusters, verbose=0)
    kmeans.fit(FVS)
    output = kmeans.predict(FVS)
    count = Counter(output)
    least_index = min(count, key=count.get)
    change_map = np.reshape(output, (new_size[1] - 2*r, new_size[0] - 2*r))
    return least_index, change_map


# Wrapper function
def runner(before="A.PNG", after="B.PNG", diff="D.PNG", change="C.PNG",
            block_size=5, erode_size=3, PCA_dimen=None):
    """
    Function that interfaces between the reading/saving of images and the actual PCA-KMC algorithm.

    :param before: The filepath (string) to the before image, with image file extension
    :param after: The filepath (string) to the after image, with image file extension
    :param diff: The filepath (string) for the created diff image, with image file extension
    :param change: The filepath (string) for the created changemap image, with image file extension
    :param block_size: Block size (h×h) - must be odd and smaller than sqrt(longest dimension)
    :param erode_size: Erosion size to clean the change map
    :param PCA_dimen: Number of PCA dimensions to consider, set to None for default
    :returns: Nothing, but the image files are just saved in the diff and change filepaths
    """

    # Run quantization
    quantize(img=before,
             img_quant_kmean=before.replace("A", "A_kmean"),
             img_quant_random=before.replace("A", "A_random"))
    quantize(img=after,
             img_quant_kmean=after.replace("B", "B_kmean"),
             img_quant_random=after.replace("B", "B_random"))

    # Run normalization
    normalize(before=before.replace("A", "A_kmean"),
              after=after.replace("B", "B_kmean"),
              after_norm=after.replace("B", "B_kmean_norm"))

    # Load images
    imgA = cv2.imread(before.replace("A", "A_kmean"))  # t1
    imgB = cv2.imread(after.replace("B", "B_kmean_norm"))  # t2
    imgD, imgC = pcakmc(imgA=imgA, imgB=imgB,
                        block_size=block_size, erode_size=erode_size, PCA_dimen=PCA_dimen)

    # Save / Display the difference image
    # cv2.imshow("diff_image", diff_image.astype(np.uint8))
    cv2.imwrite(diff, imgD)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # Display the cleaned change map
    # cv2.imshow("cleanChangeMap", cleanChangeMap)
    cv2.imwrite(change, imgC)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


# Actual PCA-KMC algorithm - called by runner()
def pcakmc(imgA, imgB, block_size, erode_size, PCA_dimen):
    """
    The actual PCA-KMC algorithm
    This implementation is based on the following paper:
    Unsupervised Change Detection in Satellite Images Using Principal Component Analysis and k-Means Clustering
    (Turgay Celik, 2009)
    http://ieeexplore.ieee.org/abstract/document/5196726

    :param imgA: An 3D R×C×K array, where R = # of pixel rows, C = # of pixel columns and K = # of channels
    :param imgB: An 3D R×C×K array, where R = # of pixel rows, C = # of pixel columns and K = # of channels
    :param block_size: Block size (h×h) - must be odd and smaller than sqrt(longest dimension)
    :param erode_size: Erosion size to clean the change map
    :param PCA_dimen: Number of PCA dimensions to consider, set to None for default
    :returns:
        imgD - An 3D R×C×K array corresponding to the diff image
        imgC - An 3D R×C×K array corresponding to the changemap image
    """

    if PCA_dimen is not None:
        assert PCA_dimen <= block_size ** 2
    assert block_size % 2 != 0

    # Params
    h = block_size
    """Block size (h×h) - must be odd and smaller than sqrt(longest dimension)"""
    r = int((h - 1) / 2)
    """Block half-size"""
    c = erode_size
    """Erosion size"""

    # Convert to grayscale
    image1 = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
    image2 = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)

    # Initialise a new size to standardise dimensions between the two images
    new_size = np.asarray(image1.shape)

    # Resize image into a multiple of block size h
    if image1.shape[0] % h != 0:
        new_size[0] = int(image1.shape[0] / h) * h
    if image1.shape[1] % h != 0:
        new_size[1] = int(image1.shape[1] / h) * h

    # Obtain difference image, i.e. their absolute intensity value differences
    # Here the difference image will have pixel values different for the regions where change is noticed when compared
    # to the rest of the image.
    image1 = cv2.resize(image1, (new_size[1], new_size[0])).astype(int)  # BUG FIX
    image2 = cv2.resize(image2, (new_size[1], new_size[0])).astype(int)  # BUG FIX
    diff_image = abs(image1 - image2)  # The difference image X_d = |X_2 - X_1|

    # Find mean-normalised vector set
    vector_set, mean_vec = find_vector_set(diff_image, new_size, h)

    # Perform PCA to get eigenvector space (EVS)
    # Alternatively to obtain the eigenvector space,
    # w, v = np.linalg.eig(vector_set)
    # can also be used, where w is the eigenvalues and v is the eigen vectors.
    if PCA_dimen is None or PCA_dimen == h ** 2:  # i.e. no dimensionality reduction
        pca = PCA(n_components=h**2)
        pca.fit(vector_set)
    else:  # TODO Future dimensionality reduction goes under find_FVS
        pca = PCA(n_components=PCA_dimen)
        pca.fit_transform(vector_set)
        raise NotImplementedError
    EVS = pca.components_  # eigenvectors are already sorted in order of decreasing eigenvalues
    # eigenvalues = pca.explained_variance_  # unhide to get eigenvalues

    # Find FVS and clusters
    FVS = find_FVS(EVS, diff_image, mean_vec, new_size, r)
    clusters = 2
    least_index, change_map = clustering(FVS, clusters, new_size, r)

    # Set unchanged cluster pixels to black
    change_map[change_map == least_index] = 255
    change_map = change_map.astype(np.uint8)

    # BUG FIX Patch to flip and rotate image at the end (TODO find better way of doing this)
    change_map = cv2.flip(change_map, 0)
    change_map = cv2.rotate(change_map, cv2.cv2.ROTATE_90_CLOCKWISE)

    # Erode/clean the change map
    kernel = np.ones((c, c), np.uint8)
    cleanChangeMap = cv2.erode(change_map, kernel)

    return diff_image, cleanChangeMap


if __name__ == "__main__":
    # Global parameters
    h = 5
    """Block size (h×h) - must be odd and smaller than sqrt(longest dimension)"""
    c = 3
    """Erosion size"""

    """
    README - TEST DATA REQUIREMENTS
    All image pairs should be located:
    - under a subfolder named "test" (or "content" for Colab)
    - under a subfolder numbered "0", "1", "2" onwards
    All image pairs should be named as:
    - "A" for before image
    - "B" for after image
    File extensions does not matter as long as:
    - A and B have the same file extensions
    - It is either .tif, .png, .jpg or .bmp format
    """

    runner(before="A.PNG",
            after="B.PNG",
            diff="D.PNG",
            change="C.PNG")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)
# Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)
# Recall
from sklearn.metrics import recall_score
recall_score(y_true, y_pred, average=None)
# Precision
from sklearn.metrics import precision_score
precision_score(y_true, y_pred, average=None)